# Stress gradient calculation

This demo shows the stress gradient calculation module. A gradient is calculated by fitting a plane into a node and its neighbor nodes of an FEM mesh.

In [ ]:

import numpy as np
import pandas as pd

import pylife.stress.equistress
import pylife.stress.gradient
import pylife.utils.meshplot


Read in demo data and add the stress tensor dimensions for the third dimension.

In [ ]:
filename = 'plate_with_hole.h5'

stress = pd.read_hdf(filename, 'node_data')
stress['S13'] = np.zeros_like(stress['S11'])
stress['S23'] = np.zeros_like(stress['S11'])

Calculate and add von Mises stress

In [ ]:
stress['mises'] = stress.equistress.mises()

Calculate stress gradient on von Mises stress

In [ ]:
grad = stress.gradient.gradient_of('mises')
grad